## Example Usage of CwmsDataStore and DssDataStore Classes ##

### Setup ###

In [6]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))

from typing import cast

from hec import CwmsDataStore, DssDataStore, HecTime, TimeSeries

### Catalog/Retrieve CWMS Time Series and Store to HEC-DSS File ###

In [7]:
db = CwmsDataStore.open(office="SWT")
# can also use:
#   db = CwmsDataStore.open() or db = CwmsDataStore.open(<api_url>)
#   db.office = "SWT"
dss = DssDataStore.open("T:/test_dss.dss", read_only=False)
# can also use
#   dss = DssDataStore.open(<filename>)
#   dss.is_read_only = False
end_time = HecTime.now().label_as_time_zone("local")
start_time = cast(HecTime, (end_time - "P1M"))
catalog = db.catalog(
    "timeseries", pattern="keys*elev*ccp-rev", fields="name,latest-time"
)
db.time_window = (
    start_time.convert_to_time_zone("UTC"),
    end_time.convert_to_time_zone("UTC"),
)
latest_times = {}
print("\nLatest Time\t\tTime Series ID")
for catalog_item in catalog:
    tsid, latest_time_str = catalog_item.split("\t")
    print(f"{latest_time_str}\t{tsid}")
    latest_times[tsid] = HecTime(latest_time_str).convert_to_time_zone("local")
print("\nRetrieving Time Series and Storing to DSS")
for tsid in latest_times:
    if latest_times[tsid] >= start_time:
        ts = cast(TimeSeries, db.retrieve(tsid)).convert_to_time_zone("local")
        print(ts)
        ts.version = f"CWMS-{ts.version}"
        dss.store(ts)
    else:
        print(f"No data in time window for {tsid}")
db.close()
print("\nDSS Catalog")
for dataset_name in dss.catalog(
    "timeseries", pattern="//keys/elev*/cwms-ccp-rev/", condensed=True
):
    print(dataset_name)
dss.close()


Latest Time		Time Series ID
2025-06-03T04:00:00Z	KEYS.Elev-Tailwater.Inst.1Hour.0.Ccp-Rev
2025-06-03T02:00:00Z	KEYS.Elev-Tailwater.Inst.30Minutes.0.Ccp-Rev
2025-06-03T04:00:00Z	KEYS.Elev.Inst.1Hour.0.Ccp-Rev
2019-11-01T00:00:00Z	KEYS.Elev.Inst.1Month.0.Ccp-Rev
2025-06-03T02:00:00Z	KEYS.Elev.Inst.30Minutes.0.Ccp-Rev

Retrieving Time Series and Storing to DSS
KEYS.Elev-Tailwater.Inst.1Hour.0.Ccp-Rev 744 values in ft
KEYS.Elev-Tailwater.Inst.30Minutes.0.Ccp-Rev 1487 values in ft
KEYS.Elev.Inst.1Hour.0.Ccp-Rev 744 values in ft
No data in time window for KEYS.Elev.Inst.1Month.0.Ccp-Rev
KEYS.Elev.Inst.30Minutes.0.Ccp-Rev 1487 values in ft

DSS Catalog
//KEYS/Elev-Tailwater/01May2025-01Jun2025/1Hour/CWMS-Ccp-Rev/
//KEYS/Elev-Tailwater/01May2025-01Jun2025/30Minute/CWMS-Ccp-Rev/
//KEYS/Elev/01May2025-01Jun2025/1Hour/CWMS-Ccp-Rev/
//KEYS/Elev/01May2025-01Jun2025/30Minute/CWMS-Ccp-Rev/
